# Some Preparations
http://www.turbare.net/transl/scipy-lecture-notes/advanced/mathematical_optimization/index.html

In [1]:
import numpy as np
from numpy.linalg import inv
epsilon = 0.00005
def get_dev(func, x,i): # Be sure to use float type array
    x_new = x.copy()
    x_new[i] = x_new[i]+epsilon
    return (func(x_new)-func(x))/epsilon

def get_dev_2nd(func, x,i,j):
    if (i !=j):
        x_new_ij, x_new_i,x_new_j = x.copy(),x.copy(),x.copy()
        x_new_ij[i],x_new_ij[j],x_new_i[i], x_new_j[j]= x[i]+epsilon,x[j]+epsilon,x[i]+epsilon,x[j]+epsilon
        results = (func(x_new_ij)+func(x)-func(x_new_i)-func(x_new_j))/(epsilon**2)
    else:
        x_new_plus, x_new_minus = x.copy(), x.copy()
        x_new_plus[i] = x[i]+epsilon
        x_new_minus[i] = x[i] - epsilon
        results = (func(x_new_plus)+func(x_new_minus)-2*func(x))/(epsilon**2)
        
    return results

# example function
from functools import reduce
def calc1(numbers):
    return sum ([ (n-1)**2 for n in numbers   ])

def calc2(numbers):
    return reduce(lambda a, b: a**0.5*b, numbers)

def calc3(numbers):
    x,y,z= numbers[0],numbers[1],numbers[2]
    return (x-1)**2 + (y+5)**2 + (z-2)**2 + x*y*z

def calc4(numbers):
    x,y,z= numbers[0],numbers[1],numbers[2]
    return -(x-1)**2 -(y+5)**2 - x*z*y - (z-2)**2


# Deepest Descent Method
Set the gradient as the direction when exploring the local miminal point.

In [1]:
def ddm(func, x):
    adjust = 0.15
    x_iter=x
    n_var = len(x)
    iter_time = 1
    while ( 1 ):
        jacob = [get_dev(func , x_iter, i) for i in range(0,n_var)]
        x_iter_new = list (  map (lambda a, b: a-b*adjust, x_iter,jacob ) )
        iter_time = iter_time + 1
        error = sum( list ( map (lambda a, b: (a-b)**2, x_iter_new, x_iter ) ) )
        #print(jacob, error)
        if (error<0.00000005 or iter_time >500):
            break
        x_iter = x_iter_new
    return x_iter

In [5]:
ddm (calc,[2,3,-4])

[0.9998841403886821, 1.0000182472506487, 0.9990794992168831]

# Newton method
Deep Descent use first-order derivative as the direction, which may lead to a low speed of convergence. Newton method considers second-order derivatives, therefore it has a higher speed of convergence.

In [27]:
def newton(func, x):
    n = len(x)
    adjust = 0.15
    x_iter = x
    iter_time = 1
    while(1):
        jacob =np.array([ get_dev(func, x_iter,i) for i in range(0,n) ] ).T
        hes_list = [ get_dev_2nd(func, x_iter,i,j) for i in range(0,n) for j in range(0,n)]
        d = np.dot(inv(np.array(hes_list).reshape(n,n)),jacob).tolist()
        x_iter_new = list (  map (lambda a, b: a-b*adjust, x_iter,d ) )
        iter_time = iter_time + 1
        error = sum( list ( map (lambda a, b: (a-b)**2, x_iter_new, x_iter ) ) )
        #print(x_iter, error)
        if (error<0.000000005 or iter_time >500):
            break
        x_iter = x_iter_new
    return x_iter


In [28]:
newton(calc3, [2.0,3.0,4.0])

[-2.9531665028331084, 1.7334606340700551, 4.560039136918094]

# Quasi-Newton Method

In [25]:
def quasi_newton(func, x, alpha = 0.05, max_iter =500, tol = 1e-5):
    # let alpha as small as possible !!!
    n = len(x)
    x_iter = x
    iter_time = 1
    B = np.eye(n)
    while(1):
        # currently, x_iter is list
        jacob =np.matrix([get_dev(func, x_iter,i) for i in range(0,n)]).T
        d = - np.dot(inv(B),jacob)
        # Turn the x_iter into a matrix
        x_iter = np.matrix(x_iter).T
        x_iter_new = x_iter + alpha * d
        iter_time = iter_time + 1
        error = np.linalg.norm (x_iter_new- x_iter )
        if (error< tol or iter_time >max_iter):
            break
        # Now calculate the new B
        s = x_iter_new - x_iter
        # Turn the x_iter_new back into list
        x_iter_new = [ x_iter_new.item(i) for i in range(0,n)]
        jacob_new = np.matrix([ get_dev(func, x_iter_new,i) for i in range(0,n) ] ).T
        y = jacob_new - jacob
        B = B-np.dot(np.dot(B,s), np.dot(B,s).T)/np.dot( np.dot(s.T,B),s )+np.dot(  y ,  y.T)/(np.dot(s.T,y))
       
        x_iter = x_iter_new
    
    x_iter = np.array(x_iter)
    x_iter = [ x_iter[i][0] for i in range(0,n)]
    return x_iter

In [26]:
quasi_newton(calc3 ,[2.0,3.0,4.0])

[-2.9531309635034315, 1.7337718808721017, 4.560130926161302]